In [25]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd 

dataset = load_dataset("sms_spam")
dataset = dataset.rename_column("sms", "text")
dataset = dataset['train']

processed_text = dataset['text']
display(type(processed_text))
processed_text = [s.replace('\n', '') for s in processed_text]

dataset = dataset.remove_columns(["text"])
display(dataset)
dataset = dataset.add_column("text", processed_text)
display(dataset)
dataset = dataset.shuffle(seed=42).select([i for i in list(range(5500))])


train_ds = Dataset.from_dict(dataset[0:4500])
validation_ds = Dataset.from_dict(dataset[4500:5000])
test_ds = Dataset.from_dict(dataset[5000:5500])


display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Reusing dataset sms_spam (C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.75it/s]


list

Dataset({
    features: ['label'],
    num_rows: 5574
})

Dataset({
    features: ['label', 'text'],
    num_rows: 5574
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\sms_spam\plain_text\1.0.0\53f051d3b5f62d99d61792c91acefe4f1577ad3e4c216fb0ad39e30b9f20019c\cache-1dabca71be75cbef.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

label                                               text
0         1  sports fans - get the latest sports news str* ...
1         0  It's justbeen overa week since we broke up and...
2         0      Not directly behind... Abt 4 rows behind ü...
3         0  Haha, my legs and neck are killing me and my a...
4         0  Me too baby! I promise to treat you well! I be...
...     ...                                                ...
4495      1  Free entry in 2 a wkly comp to win FA Cup fina...
4496      1  I am hot n horny and willing I live local to y...
4497      0  Me too watching surya movie only. . .after 6 p...
4498      0  Good evening Sir, hope you are having a nice d...
4499      0  Yar lor actually we quite fast... Cos da ge sl...

[4500 rows x 2 columns]

In [26]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [27]:
#creating antonyms list using wordnet
def antonyms_list_creation(target_word):
    antonyms_list = []
    try:
        for ann in wordnet.synsets(target_word):
            for lm in ann.lemmas():
                if lm.antonyms():
                    antonyms_list.append(lm.antonyms()[0].name())
        #print (set(antonyms_list))
        antonyms_list = list(set(antonyms_list))
        #transform _ to space
        antonyms_list = [s.replace('_', ' ') for s in antonyms_list]


        antonyms_list
        antonyms_list = [s for s in antonyms_list if ' ' not in s]
        
    except Exception as e:
        pass
    
    return antonyms_list

In [28]:
from textattack.transformations import WordSwap
class Antonyms_WordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        antonyms_list = antonyms_list_creation(word)
        
        return antonyms_list

In [29]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter


transformation = Antonyms_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en')
            ]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!


In [30]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [31]:


print(tim/3600)




0.11187335552777777


In [32]:
display(results_list)

[['sports fans - end the latest sports news str* 2 ur stabile 1 wk FREE PLUS a FREE atonicity Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we repair up and already our brains are going to mush!"],
 ['Not indirectly behind... Abt 4 rows behind ü...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to beginning the night with a burn, think I could swing by in unlike an hour?'],
 ['Me too baby! I promise to treat you well! I bet you will disclaim good care of me...'],
 ['Enjoy ur life. . ill night'],
 ['Sday only joined.so training we stop today:)'],
 ['Prof: you have fail in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U sleeping all sunset?'],
 ['refuse your lunch and come quickly and open the door:)'],
 ['Cramps continue. Going back to sleep'],
 ["MAKE SURE ALEX KNOWS HIS BIRTHDAY IS OVER IN FIFTEEN MINUTES AS FAR AS YOU'RE unconcerned

In [33]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['sports fans - get the latest sports news str* 2 ur mobile 1 wk FREE PLUS a FREE TONE Txt SPORT ON to 8007 www.getzed.co.uk 0870141701216+ norm 4txt/120p '],
 ["It's justbeen overa week since we broke up and already our brains are going to mush!"],
 ['Not directly behind... Abt 4 rows behind ü...'],
 ['Haha, my legs and neck are killing me and my amigos are hoping to end the night with a burn, think I could swing by in like an hour?'],
 ['Me too baby! I promise to treat you well! I bet you will take good care of me...'],
 ['Enjoy ur life. . Good night'],
 ['Sday only joined.so training we started today:)'],
 ['Prof: you have passed in all the papers in this sem congrats . . . . Student: Enna kalaachutaarama..!! Prof:???? Gud mrng!'],
 ['Hey whats up? U sleeping all morning?'],
 ['Have your lunch and come quickly and open the door:)'],
 ['Cramps stopped. Going back to sleep'],
 ["MAKE SURE ALEX KNOWS HIS BIRTHDAY IS OVER IN FIFTEEN MINUTES AS FAR AS YOU'RE CONCERNED"],
 ['"GRAN ONLYFO

In [34]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


original_text  \
0     sports fans - get the latest sports news str* ...   
1     It's justbeen overa week since we broke up and...   
2         Not directly behind... Abt 4 rows behind ü...   
3     Haha, my legs and neck are killing me and my a...   
4     Me too baby! I promise to treat you well! I be...   
...                                                 ...   
4495  Free entry in 2 a wkly comp to win FA Cup fina...   
4496  I am hot n horny and willing I live local to y...   
4497  Me too watching surya movie only. . .after 6 p...   
4498  Good evening Sir, hope you are having a nice d...   
4499  Yar lor actually we quite fast... Cos da ge sl...   

                                         perturbed_text  
0     sports fans - end the latest sports news str* ...  
1     It's justbeen overa week since we repair up an...  
2       Not indirectly behind... Abt 4 rows behind ü...  
3     Haha, my legs and neck are killing me and my a...  
4     Me too baby! I promise to treat you well! I be...  
...                                                 ...  
4495  Free entry in 2 a wkly comp to lose FA Cup fin...  
4496  I am hot n horny and disinherit I live nationa...  
4497  Me too watching surya movie only. . .after 6 p...  
4498  bad evening Sir, hope you are having a nice da...  
4499  Yar lor actually we quite slow... Cos da ge sl...  

[4500 rows x 2 columns]

In [35]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

original_text  \
0     sports fans - get the latest sports news str* ...   
1     It's justbeen overa week since we broke up and...   
2         Not directly behind... Abt 4 rows behind ü...   
3     Haha, my legs and neck are killing me and my a...   
4     Me too baby! I promise to treat you well! I be...   
...                                                 ...   
3762                   Yeah that's the impression I got   
3763  Free entry in 2 a wkly comp to win FA Cup fina...   
3764  I am hot n horny and willing I live local to y...   
3765  Good evening Sir, hope you are having a nice d...   
3766  Yar lor actually we quite fast... Cos da ge sl...   

                                         perturbed_text  
0     sports fans - end the latest sports news str* ...  
1     It's justbeen overa week since we repair up an...  
2       Not indirectly behind... Abt 4 rows behind ü...  
3     Haha, my legs and neck are killing me and my a...  
4     Me too baby! I promise to treat you well! I be...  
...                                                 ...  
3762                 Yeah that's the impression I leave  
3763  Free entry in 2 a wkly comp to lose FA Cup fin...  
3764  I am hot n horny and disinherit I live nationa...  
3765  bad evening Sir, hope you are having a nice da...  
3766  Yar lor actually we quite slow... Cos da ge sl...  

[3767 rows x 2 columns]

In [36]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\sms_spam_dataset_3runs\augmented_datasets\Antonym with Pos Verb_Noun=False\run3\dataset.csv')